
### Завдання 1: Виклик LLM з базовим промптом

Створіть можливість викликати LLM зі звичайним текстовим промптом.

Промпт має дозвляти отримати інформацію простою мовою на певну тему. В цьому завданні ми хочемо дізнатись про тему "Квантові обчислення".

Відповідь моделі повинна містити визначення, ключові переваги та поточні дослідження в цій галузі.

Обмежте відповідь до 200 символів і пропишіть в промпті аби відповідь була короткою (це зекономить Вам час і гроші на згенеровані токени).

В якості LLM можна скористатись як моделлю з HugginFace (рекомендую Mistral), так і ChatGPT4 або ChatGPT3. В обох випадках треба імпортувати потрібну "обгортку" (тобто клас, який дозволить ініціювати модель) з LangChain для виклику LLM за API, а також зчитати особистий токен з файла, наприклад, `creds.json`, який розміщений у Вас локально і Ви НЕ здаєте його в ДЗ і НЕ комітите в git 😏

Встановіть своє значення температури на свій розсуд (тут немає правильного чи неправильного значення) і напишіть, чому ви обрали саме таке значення для цього завдання.  

Запити можна робити як українською, так і англійською - орієнтуйтесь на те, де і чи хочете ви потім лишити цей проєкт і відповідна яка мова буде пасувати більше. В розвʼязках промпти - українською.

In [5]:
# from langchain.llms import HuggingFaceHub
# import json

# with open("creds.json", "r") as f:
#     creds = json.load(f)

# hf_api_token = creds["HF_API_TOKEN"]

# llm = HuggingFaceHub(
#     repo_id="mistralai/Mistral-7B-Instruct-v0.3",
#     task="text-generation",  
#     model_kwargs={"temperature": 0.7, "max_new_tokens": 200},
#     huggingfacehub_api_token=hf_api_token
# )

# prompt = (
#     "Explain quantum computing in simple terms. "
#     "Include definition, key advantages, and current research. "
#     "Limit answer to 200 characters."
# )

# response = llm(prompt)

In [13]:
# Alternative approach using HuggingFace Pipeline directly
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline
import json

# Load credentials
with open("creds.json", "r") as f:
    creds = json.load(f)
hf_api_token = creds["HF_API_TOKEN"]

# Create a pipeline directly
try:
    # Option 1: Using a local pipeline (if you have the model downloaded)
    pipe = pipeline(
        "text-generation", 
        model="microsoft/DialoGPT-small",  # Smaller model for reliability
        tokenizer="microsoft/DialoGPT-small"
    )
    llm = HuggingFacePipeline(pipeline=pipe)
    
except Exception as e:
    print(f"Local pipeline failed: {e}")
    print("Trying with HuggingFaceEndpoint and a different model...")
    
    # Option 2: Fall back to a simple, reliable model
    from langchain_huggingface import HuggingFaceEndpoint
    
    llm = HuggingFaceEndpoint(
        repo_id="gpt2",  # Simple, widely supported model
        temperature=0.7,
        max_new_tokens=200,
        huggingfacehub_api_token=hf_api_token
    )

# Create a better prompt that encourages detailed responses
prompt = (
    "Q: Can you explain quantum computing in detail?\n"
    "A: Quantum computing is a revolutionary technology that"
)

# Generate response
response = llm.invoke(prompt)
print(response)

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q: Can you explain quantum computing in detail?
A: Quantum computing is a revolutionary technology that can be used to solve any problem.


In [14]:
prompt = (
    "Article: Understanding Quantum Computing\n\n"
    "Quantum computing represents one of the most significant technological breakthroughs of our time. "
    "To understand quantum computing, we first need to know that"
)

response = llm.invoke(prompt)
print(response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Article: Understanding Quantum Computing

Quantum computing represents one of the most significant technological breakthroughs of our time. To understand quantum computing, we first need to know that it is a computer.


### Завдання 2: Створення параметризованого промпта для генерації тексту
Тепер ми хочемо оновити попередній фукнціонал так, аби в промпт ми могли передавати тему як параметр. Для цього скористайтесь `PromptTemplate` з `langchain` і реалізуйте параметризований промпт та виклик моделі з ним.

Запустіть оновлений функціонал (промпт + модел) для пояснень про теми
- "Баєсівські методи в машинному навчанні"
- "Трансформери в машинному навчанні"
- "Explainable AI"

Виведіть результати відпрацювання моделі на екран.

In [13]:
# pip install langchain
# !pip install langchain_openai

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 KB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 946.9/946.9 KB 15.4 MB/s eta 0:00:0031m15.5 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.6 MB/s eta 0:00:00m eta 0:00:01
  Attempting uninstall: openai
    Found existing installation: openai 1.79.0
    Uninstalling openai-1.79.0:
      Successfully uninstalled openai-1.79.0


In [14]:
from langchain.prompts import PromptTemplate
from langchain_openai.chat_models import ChatOpenAI
import os

# Ensure the environment variable is set before initializing ChatOpenAI
# Replace "YOUR_OPENAI_API_KEY" with your actual API key
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

template = "Explain the following concept in simple terms:\n{topic}"
prompt = PromptTemplate(template=template, input_variables=["topic"])

In [15]:
chat = ChatOpenAI()

topics = ["Artificial Intelligence", "Machine Learning", "Deep Learning"]
explanations = {}

for topic in topics:
  prompt_string = prompt.format(topic=topic)
  response = chat.invoke(prompt_string)
  explanations[topic] = response.content

for topic, explanation in explanations.items():
  print(f"Explanation for {topic}:\n{explanation}\n---")

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}



### Завдання 3: Використання агента для автоматизації процесів
Створіть агента, який допоможе автоматично шукати інформацію про останні наукові публікації в різних галузях. Наприклад, агент має знайти 5 останніх публікацій на тему штучного інтелекту.

**Кроки:**
1. Налаштуйте агента типу ReAct в LangChain для виконання автоматичних запитів.
2. Створіть промпт, який спрямовує агента шукати інформацію в інтернеті або в базах даних наукових публікацій.
3. Агент повинен видати список публікацій, кожна з яких містить назву, авторів і короткий опис.

Для взаємодії з пошуком там необхідно створити `Tool`. В лекції ми використовували `serpapi`. Можна продовжити користуватись ним, або обрати інше АРІ для пошуку (вони в тому числі є безкоштовні). Перелік різних АРІ, доступних в langchain, і орієнтир по вартості запитів можна знайти в окремому документі [тут](https://hannapylieva.notion.site/API-12994835849480a69b2adf2b8441cbb3?pvs=4).

Лишаю також нижче приклад використання одного з безкоштовних пошукових АРІ - DuckDuckGo (не потребує створення токена!)  - можливо він вам сподобається :)


In [4]:
!pip install -q langchain_community duckduckgo_search

In [5]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

# search.invoke("Obama's first name?")

In [6]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain.tools import DuckDuckGoSearchResults

llm = OpenAI(temperature=0)

search_tool = DuckDuckGoSearchResults(max_results=5)

tools = [
    Tool(
        name="Search",
        func=search_tool.run,
        description="Use this tool to find recent scientific publications on a given topic. "
                    "Return title, authors, and short description if available."
    )
]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

query = "Find 5 recent scientific publications on artificial intelligence, including title, authors, and a short description."

result = agent.run(query)
print(result)


/tmp/ipykernel_36859/2691837421.py:6: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0)


ValidationError: 1 validation error for OpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'temperature': 0, 'model...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error



### Завдання 4: Створення агента-помічника для вирішення бізнес-задач

Створіть агента, який допомагає вирішувати задачі бізнес-аналітики. Агент має допомогти користувачу створити прогноз по продажам на наступний рік враховуючи рівень інфляції і погодні умови. Агент має вміти використовувати Python і ходити в інтернет аби отримати актуальні дані.

**Кроки:**
1. Налаштуйте агента, який працюватиме з аналітичними даними, заданими текстом. Користувач пише

```
Ми експортуємо апельсини з Бразилії. В 2021 експортували 200т, в 2022 - 190т, в 2023 - 210т, в 2024 який ще не закінчився - 220т. Зроби оцінку скільки ми зможемо експортувати апельсинів в 2025 враховуючи погодні умови в Бразилії і попит на апельсини в світі виходячи з економічної ситуації.
```

2. Створіть запит до агента, що містить чітке завдання – видати результат бізнес аналізу або написати, що він не може цього зробити і запит користувача (просто може бути все одним повідомлленням).

3. Запустіть агента і проаналізуйте результати. Що можна покращити?


In [10]:
# pip install langchain openai duckduckgo-search pandas matplotlib

from langchain.agents import initialize_agent, Tool, AgentType
from langchain.llms import OpenAI
from langchain.tools import DuckDuckGoSearchResults
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import PythonAgent
from langchain.tools.python.tool import PythonREPLTool

llm = OpenAI(temperature=0)

search_tool = DuckDuckGoSearchResults(max_results=3)
search = Tool(
    name="Search",
    func=search_tool.run,
    description="Use this tool to get latest data about weather in Brazil, orange demand, inflation, or any relevant economic info."
)

python_tool = PythonREPLTool()

tools = [search, python_tool]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

user_input = """
We export oranges from Brazil. In 2021 we exported 200t, in 2022 - 190t, in 2023 - 210t, 
and in 2024 (not finished yet) - 220t. 
Make an estimate of how many oranges we might export in 2025 
taking into account weather conditions in Brazil and global orange demand based on the economic situation.
"""

result = agent.run(user_input)

print("=== BUSINESS ANALYSIS RESULT ===")
print(result)


ValueError: '/home/liubov/.local/lib/python3.10/site-packages/langchain/agents/agent_toolkits' is not in the subpath of '/home/liubov/.local/lib/python3.10/site-packages/langchain_core' OR one path is relative and the other is absolute.

In [9]:
import os

# Зчитуємо файл рядок за рядком
with open("env.txt") as f:
    for line in f:
        line = line.strip()
        if line.startswith("export "):
            # Розділяємо на ключ і значення
            key_value = line[len("export "):].split("=", 1)
            if len(key_value) == 2:
                key, value = key_value
                value = value.strip('"')  # прибрати лапки
                os.environ[key] = value  # додаємо у змінні оточення

# Тепер можна використовувати як змінну оточення
api_key = os.getenv("OPENAI_API_KEY")